In [1]:
"""
Inpainting CMB example
"""

'\nInpainting CMB example\n'

In [2]:
import numpy as np, sys, os, tools, scipy as sc, inpaint
import flatsky, tools
#%load_ext autoreload
#%autoreload 2

#%pylab notebook
%matplotlib inline
from pylab import *
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = [r'\boldmath']
rcParams["figure.facecolor"] = 'white'
cmap = cm.RdYlBu

/var/folders/08/mkcy0rls72j2q09krs7qx2jw0000gn/T/ipykernel_52547/1963589971.py:9: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = [r'\boldmath']


In [3]:
rcParams['figure.dpi'] = 150
rcParams['font.family'] = 'Times new roman'

In [13]:
#params or supply a params file
dx = 1.0
boxsize_am = 200. #boxsize in arcmins
nx = int(boxsize_am/dx)
mapparams = [nx, nx, dx, dx]
x1,x2 = -nx/2. * dx, nx/2. * dx
verbose = 0
#lmax = 10000
#el = np.arange(lmax)

#beam and noise levels
noiseval = 1e-3 #2.0 #uK-arcmin
beamval = 1.4 #arcmins

#CMB power spectrum
#Cls_file = 'data/Cl_Planck2018_camb.npz'
Cls_file = 'data/output_planck_r_0.0_2015_cosmo_lensedCls.dat'
Tcmb = 2.73

#for inpainting
noofsims = 1000
mask_radius_inner = 4.0 #arcmins
mask_radius_outer = 20.0 #arcmins
mask_inner = 0  #If 1, the inner region is masked before the LPF. Might be useful in the presence of bright SZ signal at the centre.

In [14]:
#get ra, dec or map-pixel grid
ra = np.linspace(x1,x2, nx) #arcmins
dec = np.linspace(x1,x2, nx) #arcmins
ra_grid, dec_grid = np.meshgrid(ra,dec)

In [15]:
#read Cls now
'''
if (0):
    #els, Dls = np.loadtxt(Cls_file, usecols = [0,1], unpack = 1)
    Cls_rec = np.load(Cls_file, allow_pickle=1, mmap_mode='r')
    #print(Cls_rec.files);sys.exit()
    el = Cls_rec['L']
    cl = Cls_rec['TT']* 1e12 #Cls in uK

    #Dls_fac = els * (els + 1) / 2 / np.pi
    #Cls = Cls * Dls_fac
'''
#read Cls now
el, dl_tt, dl_ee, dl_bb, dl_te  = np.loadtxt(Cls_file, unpack = 1)
dl_all = np.asarray( [dl_tt, dl_ee, dl_bb, dl_te] )
dl_fac = el * (el + 1) / 2 / np.pi
cl_all = ( Tcmb**2. * dl_all ) / ( dl_fac )
cl_all = cl_all * 1e12
cl_tt, cl_ee, cl_bb, cl_te = cl_all #Cls in uK
cl = cl_tt
cl_dic = {}
cl_dic['TT'] = cl_tt

#loglog(el, cl_tt)
print(len(el))

19963


In [16]:
#get beam and noise
bl = tools.get_bl(beamval, el, make_2d = 1, mapparams = mapparams)
nl = tools.get_nl(noiseval, el)
nl_dic = {}
nl_dic['T'] = nl
print(nl_dic)

{'T': array([8.46159499e-14, 8.46159499e-14, 8.46159499e-14, ...,
       8.46159499e-14, 8.46159499e-14, 8.46159499e-14])}


Text(0.5, 0, 'Multipole $\\ell$')

In [17]:
#plot
ax =subplot(111, yscale = 'log', xscale = 'log')
plot(el, cl_tt, 'k-', label = r'CMB')
plot(el, nl, 'r-', label = r'Noise')
legend(loc = 1)
ylim(1e-10, 1e4)
ylabel(r'$C_{\ell}$')
xlabel(r'Multipole $\ell$')

Text(0.5, 1.0, 'CMB + Noise')

In [18]:
#create CMB and convolve with beam
cmb_map = flatsky.make_gaussian_realisation(mapparams, el, cl, bl = bl)

#noise map
noise_map = flatsky.make_gaussian_realisation(mapparams, el, nl)
sim_map = cmb_map + noise_map
subplot(131);imshow(cmb_map, extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'CMB')
subplot(132);imshow(noise_map, extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'Noise')
subplot(133);imshow(sim_map, extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'CMB + Noise')

In [19]:
#get covariance for inpainting
sigma_dic = inpaint.get_covariance(ra_grid, dec_grid, mapparams, el, cl_dic, bl, nl_dic, noofsims, mask_radius_inner, mask_radius_outer, low_pass_cutoff = 1)
print(sigma_dic.keys())


	calculating the covariance from simulations for inpainting
2698

		generating 1000 sims

			invert sigma_22 matrix (1188,1188) now

		covariance obtained
dict_keys(['sigma_22_inv', 'sigma_12'])


In [20]:
#perform inpainting
sim_map_dic = {}
sim_map_dic['T'] = sim_map
print(sim_map_dic.keys())
cmb_inpainted_map, sim_map_inpainted, sim_map_filtered = inpaint.inpainting(sim_map_dic, ra_grid, dec_grid, mapparams, el, cl_dic, bl, nl_dic, noofsims, mask_radius_inner, mask_radius_outer, low_pass_cutoff = 1, mask_inner = mask_inner, sigma_dic = sigma_dic)

dict_keys(['T'])


In [21]:
clf()
subplots_adjust(hspace = 0.25, wspace = 0.25)
tquarr = ['T', 'Q', 'U']
tqulen = len(cmb_inpainted_map)
tr, tc = 2, 3
sbpl = 1
fsval = 8
for tqucntr in range(0,1): #plot T first
    subplot(tr, tc, sbpl);imshow(sim_map_filtered[tqucntr], extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'Input map: %s' %(tquarr[tqucntr]), fontsize = fsval)
    ylabel('Y [arcmins]')
    sbpl+=1
    subplot(tr, tc, sbpl);imshow(sim_map_inpainted[tqucntr], extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'Inpainted map: %s' %(tquarr[tqucntr]), fontsize = fsval)
    sbpl+=1
    residual = sim_map_filtered - sim_map_inpainted
    subplot(tr, tc, sbpl);imshow(residual[tqucntr], extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'Residual: %s' %(tquarr[tqucntr]), fontsize = fsval)
    sbpl+=1

    #zooming in
    s, e = 95, 105
    x1, x2 = -(e-s)*dx/2.,(e-s)*dx/2.
    circle = Circle((0, 0), radius = mask_radius_inner, facecolor='None', edgecolor = 'black', lw = 1., ls = '-');
    ax=subplot(tr, tc, sbpl);imshow(sim_map_filtered[tqucntr, s:e, s:e], extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'Input map: %s' %(tquarr[tqucntr]), fontsize = fsval)
    sbpl+=1
    ax.add_artist(circle)
    xlabel('X [arcmins]'); ylabel('Y [arcmins]')

    circle = Circle((0, 0), radius = mask_radius_inner, facecolor='None', edgecolor = 'black', lw = 1., ls = '-');
    ax=subplot(tr, tc, sbpl);imshow(sim_map_inpainted[tqucntr, s:e, s:e], extent = [x1,x2,x1,x2], cmap = cmap); colorbar(); title(r'Inpainted map: %s' %(tquarr[tqucntr]), fontsize = fsval)
    sbpl+=1
    ax.add_artist(circle)
    xlabel('X [arcmins]')

    residual_flatten = (residual[0, s:e, s:e].flatten())*1e3
    minval, maxval, delta = -1., 1., 0.1
    binbin = np.arange(minval, maxval, delta)
    subplot(tr, tc, sbpl);hist(residual_flatten, bins = binbin, histtype = 'step', color = 'black'); title(r'Residual: %s'  %(tquarr[tqucntr]), fontsize = fsval)
    sbpl+=1
    xlabel(r'Residual [nK]')

show()